# ===============================================================
#  Project: Banking Insights Dashboard
#  Step 2: Data Cleaning & Preprocessing
#  Author: [Your Name]
#  Description:
#    This notebook cleans, standardizes, and merges the raw banking
#    datasets into one analysis-ready master file for Tableau.
# ===============================================================

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

In [2]:
# Load all raw data files
customers = pd.read_csv('../data/Bank_Customer_Data.csv')
accounts = pd.read_csv('../data/Bank_Account_Data.csv')
branches = pd.read_csv('../data/Bank_Branch_Data.csv')
loans = pd.read_csv('../data/Bank_Loan_Data.csv')
transactions = pd.read_csv('../data/Bank_Transacation_Data.csv')

print("✅ Data loaded successfully!")
print(f"Customers: {customers.shape}, Accounts: {accounts.shape}, Branches: {branches.shape}, Loans: {loans.shape}, Transactions: {transactions.shape}")


✅ Data loaded successfully!
Customers: (3000, 7), Accounts: (3000, 7), Branches: (150, 3), Loans: (1500, 4), Transactions: (15000, 6)


In [3]:
# Convert all column names to lowercase and replace spaces with underscores
def clean_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

datasets = [customers, accounts, branches, loans, transactions]
customers, accounts, branches, loans, transactions = [clean_columns(df) for df in datasets]

print("✅ Column names standardized!")


✅ Column names standardized!


In [4]:
# Convert date columns to datetime
date_columns = {
    'customers': ['dob'],
    'accounts': ['account_open_date'],
    'transactions': ['transcation_date']
}

for col in date_columns['customers']:
    customers[col] = pd.to_datetime(customers[col], errors='coerce')

for col in date_columns['accounts']:
    accounts[col] = pd.to_datetime(accounts[col], errors='coerce')

for col in date_columns['transactions']:
    transactions[col] = pd.to_datetime(transactions[col], errors='coerce')

print("✅ Date columns converted!")


✅ Date columns converted!


In [5]:
# Check missing values
print("Missing values before cleaning:")
print(customers.isnull().sum())

# Fill missing or invalid entries
customers.fillna({'occupation': 'Unknown'}, inplace=True)
accounts.fillna({'account_status': 'Active'}, inplace=True)
transactions['transcation_amount'].fillna(0, inplace=True)
loans['loan_amount'].fillna(0, inplace=True)

# Drop duplicate rows if any
customers.drop_duplicates(inplace=True)
accounts.drop_duplicates(inplace=True)
transactions.drop_duplicates(inplace=True)

print("✅ Missing values handled and duplicates removed!")


Missing values before cleaning:
customer_id     0
first_name      0
last_name       0
city            0
phone_number    0
occupation      0
dob             0
dtype: int64
✅ Missing values handled and duplicates removed!


C:\Users\ANANTA OJHA\AppData\Local\Temp\ipykernel_15052\1952006855.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  transactions['transcation_amount'].fillna(0, inplace=True)
C:\Users\ANANTA OJHA\AppData\Local\Temp\ipykernel_15052\1952006855.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behav

In [6]:
today = pd.Timestamp.today()
customers['customer_age'] = (today - customers['dob']).dt.days // 365


In [7]:
accounts['account_age'] = (today - accounts['account_open_date']).dt.days // 365


In [8]:
# Aggregate transaction data per account
txn_summary = transactions.groupby('account_id').agg({
    'transcation_amount': ['count', 'sum'],
    'transcation_date': 'max'
}).reset_index()

txn_summary.columns = ['account_id', 'total_transactions', 'total_transaction_amount', 'last_transaction_date']

print("✅ Transaction summary created!")
txn_summary.head()


✅ Transaction summary created!


,account_id,total_transactions,total_transaction_amount,last_transaction_date
0,A00001,14,795292,2022-10-14
1,A00002,19,1020426,2022-05-02
2,A00003,19,961033,2022-08-21
3,A00004,12,528939,2022-12-05
4,A00005,19,1006175,2023-04-18


In [9]:
# Merge customers → accounts → branches → loans → transactions
merged = accounts.merge(customers, on='customer_id', how='left')
merged = merged.merge(branches, on='branch_id', how='left')
merged = merged.merge(loans[['customer_id', 'loan_amount']], on='customer_id', how='left')
merged = merged.merge(txn_summary, on='account_id', how='left')

print("✅ All datasets merged successfully!")
print("Final shape:", merged.shape)
merged.head(3)


✅ All datasets merged successfully!
Final shape: (43686, 21)


,account_id,customer_id,branch_id,opening_balance,account_open_date,account_type,account_status,account_age,first_name,last_name,city,phone_number,occupation,dob,customer_age,branch_name,branch_state,loan_amount,total_transactions,total_transaction_amount,last_transaction_date
0,A00001,C00001,B00019,520482,2014-03-01,Savings,Pending,11,Timothy,Nelson,Davidville,1592081539,Magician,1915-09-03,110,NM00018,New Mexico,7799194.0,14,795292,2022-10-14
1,A00001,C00001,B00019,520482,2014-03-01,Savings,Pending,11,Timothy,Nelson,Davidville,1592081539,Magician,1915-09-03,110,NM00018,New Mexico,4814936.0,14,795292,2022-10-14
2,A00001,C00001,B00019,520482,2014-03-01,Savings,Pending,11,Timothy,Nelson,Davidville,1592081539,Magician,1915-09-03,110,PR00018,Puerto Rico,7799194.0,14,795292,2022-10-14


In [10]:
six_months_ago = today - pd.DateOffset(months=6)
merged['churn_flag'] = np.where(merged['last_transaction_date'] < six_months_ago, 1, 0)


In [11]:
loan_conversion = merged.groupby('branch_state')['loan_amount'].apply(lambda x: x.notnull().mean()).reset_index()
loan_conversion.columns = ['branch_state', 'loan_conversion_rate']


In [12]:
avg_balance = merged.groupby('branch_state')['opening_balance'].mean().reset_index()
avg_balance.columns = ['branch_state', 'avg_balance']


In [13]:
branch_kpi = loan_conversion.merge(avg_balance, on='branch_state', how='outer')
print("✅ Regional KPIs generated!")
branch_kpi.head()


✅ Regional KPIs generated!


,branch_state,loan_conversion_rate,avg_balance
0,Alabama,0.930108,507050.725806
1,Alaska,0.917949,466258.758974
2,Arizona,0.909278,519842.967010
3,Arkansas,0.921147,517314.792115
4,California,0.883534,512595.879518


In [14]:
merged.fillna({
    'loan_amount': 0,
    'total_transactions': 0,
    'total_transaction_amount': 0
}, inplace=True)

merged.drop_duplicates(inplace=True)
print("✅ Final cleaning complete!")


✅ Final cleaning complete!


In [15]:
# Save cleaned and merged data
customers.to_csv('../data/customers_clean.csv', index=False)
accounts.to_csv('../data/accounts_clean.csv', index=False)
transactions.to_csv('../data/transactions_clean.csv', index=False)
merged.to_csv('../data/merged_dataset.csv', index=False)
branch_kpi.to_csv('../data/branch_kpi.csv', index=False)

print("✅ Cleaned datasets exported successfully to /data folder!")


✅ Cleaned datasets exported successfully to /data folder!


In [16]:
print("Total Customers:", merged['customer_id'].nunique())
print("Total Branches:", merged['branch_id'].nunique())
print("Churn Rate:", round(merged['churn_flag'].mean() * 100, 2), "%")
print("Average Loan Amount:", round(merged['loan_amount'].mean(), 2))
print("Average Account Balance:", round(merged['opening_balance'].mean(), 2))


Total Customers: 1000
Total Branches: 50
Churn Rate: 100.0 %
Average Loan Amount: 4749896.27
Average Account Balance: 497376.94


In [17]:
summary_stats = {
    "total_customers": merged['customer_id'].nunique(),
    "total_accounts": merged['account_id'].nunique(),
    "total_branches": merged['branch_id'].nunique(),
    "total_loans": merged['loan_amount'].astype(bool).sum(),
    "avg_balance": merged['opening_balance'].mean(),
    "avg_loan_amount": merged['loan_amount'].mean(),
    "churn_rate_percent": merged['churn_flag'].mean() * 100
}

pd.DataFrame([summary_stats]).to_csv('../docs/data_cleaning_summary.csv', index=False)
print("📄 Summary stats saved to docs/data_cleaning_summary.csv")


📄 Summary stats saved to docs/data_cleaning_summary.csv


In [18]:
from IPython.display import Markdown as md

md("""
## ✅ Next Step: Exploratory Data Analysis (EDA)

You now have a clean and analysis-ready dataset:
- `merged_dataset.csv` → main file for Tableau and EDA
- `branch_kpi.csv` → branch-level KPIs (loan conversion, avg balance)
- `customers_clean.csv`, `accounts_clean.csv`, `transactions_clean.csv`

### Coming up in Step 3:
- Perform exploratory analysis (distributions, correlations)
- Identify top performing regions
- Validate churn & loan patterns visually (Matplotlib / Seaborn)
- Prepare key visuals before moving into Tableau dashboard creation
""")



## ✅ Next Step: Exploratory Data Analysis (EDA)

You now have a clean and analysis-ready dataset:
- `merged_dataset.csv` → main file for Tableau and EDA
- `branch_kpi.csv` → branch-level KPIs (loan conversion, avg balance)
- `customers_clean.csv`, `accounts_clean.csv`, `transactions_clean.csv`

### Coming up in Step 3:
- Perform exploratory analysis (distributions, correlations)
- Identify top performing regions
- Validate churn & loan patterns visually (Matplotlib / Seaborn)
- Prepare key visuals before moving into Tableau dashboard creation
